# Performs Data Quality Checks

In [14]:
import os
import unittest
import configparser
import psycopg2
import psycopg2.extras
from sql_queries import prod_tables

config = configparser.ConfigParser()
config.read(os.path.join(os.path.dirname(os.getcwd()), 'config.ini'))

['config.ini']

In [ ]:
host = config.get('RDS','host')
username = config.get('RDS','user')
password = config.get('RDS','password')
database = config.get('RDS','database')      
db = psycopg2.connect(host=host, user=username, password=password, database=database)
curr = db.cursor(cursor_factory=psycopg2.extras.DictCursor)
curr.execute("SELECT COUNT(*) FROM {}".format(table))
record_count = (self.curr).fetchone()

In [1]:
def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite)

In [23]:
class TestQualityCheck(unittest.TestCase):
    curr = None
    
    @classmethod
    def setUpClass(cls):
        # Set up db connections
        host = config.get('postgres','host')
        username = config.get('postgres','user')
        password = config.get('postgres','password')
        database = config.get('postgres','database')      
        db = psycopg2.connect(host=host, user=username, password=password, database=database)
        cls.curr = db.cursor(cursor_factory=psycopg2.extras.DictCursor)
        
    @classmethod
    def tearDownClass(cls):
        if (cls.curr):
            (cls.curr).close()
        
    def test_connection(self):
        self.assertNotEqual(self.curr, None)
        
    def test_distinct_records(self):
        for table in prod_tables:
            self.curr.execute("SELECT COUNT(*) FROM {}".format(table))
            record_count = (self.curr).fetchone()
            
            self.curr.execute("SELECT COUNT(*) FROM (SELECT DISTINCT * FROM {}) AS temp".format(table))
            distinct_record_count = (self.curr).fetchone()
            
            print("{}: Record Count [{}], Distinct Count [{}]".format(table, record_count, distinct_record_count))
            self.assertEqual(record_count, distinct_record_count)

In [24]:
run_tests(TestQualityCheck)

test_connection (__main__.TestQualityCheck) ... ok
test_distinct_records (__main__.TestQualityCheck) ... 

prod_police_shootings: Record Count [[6533]], Distinct Count [[6533]]
prod_us_cities: Record Count [[28338]], Distinct Count [[28338]]
prod_us_demographics: Record Count [[87976]], Distinct Count [[87976]]
prod_unemployment: Record Count [[3237]], Distinct Count [[3237]]


ok

----------------------------------------------------------------------
Ran 2 tests in 0.177s

OK
